<a href="https://colab.research.google.com/github/Ju-H-end/erebb-site/blob/main/Transformada_JH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy.signal import butter, lfilter, welch, stft
from scipy.ndimage import gaussian_filter1d

# Canais desejados
CHANNELS = ['F3', 'F4', 'C3', 'C4', 'P3', 'P4']

# Função para simular o carregamento de dados EEG (já que não temos os arquivos .edf)
def load_eeg_data_simulated(patient_id, sfreq=250, duration=10):
    n_samples = int(sfreq * duration)
    data = np.random.randn(len(CHANNELS), n_samples) * 1e-5  # Simula dados EEG em Volts
    info = {
        'ch_names': CHANNELS,
        'sfreq': sfreq,
        'nchan': len(CHANNELS),
        'meas_date': None
    }
    return data, info

# Funções de pré-processamento
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

def normalize_signal(data):
    return (data - np.mean(data)) / np.std(data)

def remove_artifacts_gaussian(data, sigma=2):
    return gaussian_filter1d(data, sigma=sigma)

# Função para extrair ID do paciente do nome do arquivo
def extract_patient_id(filename):
    return filename.split('_')[0]

def main():
    try:
        df = pd.read_csv('Registros_canais_desejados_epilepsy.csv')
        print("Arquivo 'Registros_canais_desejados_epilepsy.csv' carregado com sucesso.")
    except FileNotFoundError:
        print("Arquivo 'Registros_canais_desejados_epilepsy.csv' não encontrado.")
        print("Criando um DataFrame de exemplo para continuar a execução.")
        df = pd.DataFrame({
            'Arquivo': ['simulado_00007656_s010_t000.edf'],
            'Categoria': ['epilepsy'],
            'Paciente_ID': [7656]
        })

    # Obter IDs únicos de pacientes
    if not df.empty:
      patient_ids = df['Paciente_ID'].unique()
    else:
      print("DataFrame está vazio, não há pacientes para processar.")
      return # Exit if df is empty


    output_dir = 'eeg_output'
    os.makedirs(output_dir, exist_ok=True)

    all_features = []

    # Defina a duração do segmento para o gráfico segmentado (em segundos)
    segment_duration = 5

    for patient_id in patient_ids:
        print(f"Processando paciente: {patient_id}")

        # Simular carregamento de dados para o paciente
        raw_data, info = load_eeg_data_simulated(patient_id)
        sfreq = info['sfreq']

        # Iterar sobre cada canal
        for i, channel_name in enumerate(CHANNELS):
            print(f"  Processando canal: {channel_name}")
            signal = raw_data[i, :]

            # 1. Visualização do sinal bruto (inteiro)
            plt.figure(figsize=(12, 6))
            plt.plot(np.arange(len(signal)) / sfreq, signal)
            plt.title(f'Sinal Bruto Inteiro - Paciente {patient_id} - Canal {channel_name}')
            plt.xlabel('Tempo (s)')
            plt.ylabel('Amplitude (V)')
            plt.grid(True)
            plt.savefig(os.path.join(output_dir, f'paciente_{patient_id}_{channel_name}_sinal_bruto_inteiro.png'))
            plt.close()

            # 2. Visualização do sinal bruto (segmentado)
            n_segment_samples = int(sfreq * segment_duration)
            if n_segment_samples < len(signal):
              plt.figure(figsize=(12, 6))
              plt.plot(np.arange(n_segment_samples) / sfreq, signal[:n_segment_samples])
              plt.title(f'Sinal Bruto Segmentado ({segment_duration}s) - Paciente {patient_id} - Canal {channel_name}')
              plt.xlabel('Tempo (s)')
              plt.ylabel('Amplitude (V)')
              plt.grid(True)
              plt.savefig(os.path.join(output_dir, f'paciente_{patient_id}_{channel_name}_sinal_bruto_segmentado.png'))
              plt.close()
            else:
                print(f"  Sinal muito curto para segmentar ({segment_duration}s). Pulando o gráfico segmentado para o canal {channel_name}.")


            # 3. Pré-processamento
            # Filtragem Band-pass (0.5 - 30 Hz)
            filtered_signal = bandpass_filter(signal, 0.5, 30, sfreq)
            # Normalização
            normalized_signal = normalize_signal(filtered_signal)
            # Remoção de artefatos (suavização gaussiana)
            preprocessed_signal = remove_artifacts_gaussian(normalized_signal)

            # Visualização do sinal pré-processado
            plt.figure(figsize=(12, 6))
            plt.plot(np.arange(len(preprocessed_signal)) / sfreq, preprocessed_signal)
            plt.title(f'Sinal Pré-processado - Paciente {patient_id} - Canal {channel_name}')
            plt.xlabel('Tempo (s)')
            plt.ylabel('Amplitude Normalizada')
            plt.grid(True)
            plt.savefig(os.path.join(output_dir, f'paciente_{patient_id}_{channel_name}_sinal_pre_processado.png'))
            plt.close()

            # 4. Transformada de Fourier de Curto Tempo (STFT) e Extração de Características
            f, t, Zxx = stft(preprocessed_signal, sfreq, nperseg=sfreq)

            # Visualização do Heatmap da STFT
            plt.figure(figsize=(10, 6))
            plt.pcolormesh(t, f, np.abs(Zxx), shading='gouraud')
            plt.title(f'STFT Heatmap - Paciente {patient_id} - Canal {channel_name}')
            plt.xlabel('Tempo (s)')
            plt.ylabel('Frequência (Hz)')
            plt.colorbar(label='Magnitude')
            plt.ylim([0, 50]) # Limitar o eixo y para melhor visualização das bandas de interesse
            plt.savefig(os.path.join(output_dir, f'paciente_{patient_id}_{channel_name}_stft_heatmap.png'))
            plt.close()


            # Exemplo: Média e desvio padrão da magnitude dos coeficientes
            mean_stft = np.mean(np.abs(Zxx))
            std_stft = np.std(np.abs(Zxx))

            # Definir as bandas de frequência
            delta_band = (0.5, 4)
            theta_band = (4, 8)
            alpha_band = (8, 13)
            beta_band = (13, 30)
            gamma_band = (30, 100)

            # Encontrar os índices das frequências correspondentes às bandas
            delta_idx = np.where((f >= delta_band[0]) & (f <= delta_band[1]))[0]
            theta_idx = np.where((f >= theta_band[0]) & (f <= theta_band[1]))[0]
            alpha_idx = np.where((f >= alpha_band[0]) & (f <= alpha_band[1]))[0]
            beta_idx = np.where((f >= beta_band[0]) & (f <= beta_band[1]))[0]
            gamma_idx = np.where((f >= gamma_band[0]) & (f <= gamma_band[1]))[0]

            # Calcular a potência média em cada banda
            delta_power = np.mean(np.abs(Zxx[delta_idx, :])) if len(delta_idx) > 0 else 0
            theta_power = np.mean(np.abs(Zxx[theta_idx, :])) if len(theta_idx) > 0 else 0
            alpha_power = np.mean(np.abs(Zxx[alpha_idx, :])) if len(alpha_idx) > 0 else 0
            beta_power = np.mean(np.abs(Zxx[beta_idx, :])) if len(beta_idx) > 0 else 0
            gamma_power = np.mean(np.abs(Zxx[gamma_idx, :])) if len(gamma_idx) > 0 else 0


            features = {
                'Paciente_ID': patient_id,
                'Canal': channel_name,
                'Media_STFT': mean_stft,
                'Std_STFT': std_stft,
                'Potencia_Delta': delta_power,
                'Potencia_Theta': theta_power,
                'Potencia_Alpha': alpha_power,
                'Potencia_Beta': beta_power,
                'Potencia_Gamma': gamma_power
            }
            all_features.append(features)


    # Salvar resumo das características em CSV
    features_df = pd.DataFrame(all_features)
    features_df.to_csv(os.path.join(output_dir, 'caracteristicas_eeg.csv'), index=False)
    print(f"Características salvas em {os.path.join(output_dir, 'caracteristicas_eeg.csv')}")

if __name__ == '__main__':
    main()

Arquivo 'Registros_canais_desejados_epilepsy.csv' não encontrado.
Criando um DataFrame de exemplo para continuar a execução.
Processando paciente: 7656
  Processando canal: F3
  Processando canal: F4
  Processando canal: C3
  Processando canal: C4
  Processando canal: P3
  Processando canal: P4
Características salvas em eeg_output/caracteristicas_eeg.csv


In [2]:
from google.colab import files
import pandas as pd
import os

output_dir = 'eeg_output'
features_file = os.path.join(output_dir, 'caracteristicas_eeg.csv')

# Verificar se o arquivo existe antes de tentar baixar
if os.path.exists(features_file):
    print(f"Baixando o arquivo: {features_file}")
    files.download(features_file)
else:
    print(f"O arquivo {features_file} não foi encontrado.")
    print("Por favor, execute a célula anterior para gerar o arquivo.")

Baixando o arquivo: eeg_output/caracteristicas_eeg.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [7]:
from google.colab import files
import os

output_dir = 'eeg_output'
image_files = [f for f in os.listdir(output_dir) if f.endswith('.png') and 'sinal_pre_processado' in f]

if image_files:
    print("Baixando os seguintes arquivos de imagem:")
    for img_file in image_files:
        print(f"- {img_file}")
        files.download(os.path.join(output_dir, img_file))
else:
    print(f"Nenhum arquivo de imagem de sinal pré-processado encontrado na pasta '{output_dir}'.")
    print("Por favor, execute a célula anterior para gerar os gráficos.")

Baixando os seguintes arquivos de imagem:
- paciente_7656_F3_sinal_pre_processado.png


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

- paciente_7656_C4_sinal_pre_processado.png


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

- paciente_7656_C3_sinal_pre_processado.png


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>